# Setup

In [ ]:
import pandas as pd
import datetime
from pathlib import Path

## only  need these to reload utils
# import importlib
# import utils

# # After making changes to your_module_name.py, run this cell
# importlib.reload(utils)

from utils import get_todays_games, filter_data_on_change, aggregate_betting_data, get_complete_game_results, process_and_save_evaluated_bets

In [72]:
import plotly_express as px

In [65]:
    
def build_ncaa_prompt(model_version):
    try:
        df_all = pd.read_csv('../data/ncaa_bets_db.csv')
    except:
        df_all = pd.DataFrame()


    # Example usage:
    HEADERS = {
        'Authority': 'api.actionnetwork',
        'Accept': 'application/json',
        'Origin': 'https://www.actionnetwork.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    }


    # Example usage:
    sport='ncaab'


    # Get today's date object
    today = datetime.date.today()

    # Define the desired string format
    date_format = '%Y%m%d'

    # Create the list using strftime() to format the dates
    date_str_list = [
        (today + datetime.timedelta(days=0)).strftime(date_format), # Today
        (today + datetime.timedelta(days=1)).strftime(date_format), # Tomorrow
        (today + datetime.timedelta(days=2)).strftime(date_format), # The next day
        (today + datetime.timedelta(days=3)).strftime(date_format)  # The day after
    ]

    df = get_todays_games(sport,date_str_list,HEADERS)
    df['date_scraped'] = datetime.datetime.now()

    df = df.loc[df['status'] == 'scheduled']

    df_all = pd.concat([df_all,df])
    df_all['date_scraped'] = pd.to_datetime(df_all['date_scraped'])

    dimension_cols = ['game_id', 'home_team', 'away_team']
    metric_cols = ['home_money_line', 'away_money_line','total_score','home_money_line','away_money_line']
    filtered_df = filter_data_on_change(df_all, dimension_cols, metric_cols)
    print(df_all.index.size)
    print(filtered_df.index.size)

    filtered_df.to_csv('../data/ncaa_bets_db.csv', index=False)

    filtered_df['start_time_pt'] = pd.to_datetime(filtered_df['start_time_pt'])


    group_by_columns = ['game_id', 'home_team', 'away_team','start_time']
    metric_columns = [
        'num_bets', 'home_money_line', 'home_ml_ticket_pct', 'home_ml_money_pct',
        'away_money_line', 'away_ml_ticket_pct', 'away_ml_money_pct', 'total_score',
        'over_odds', 'under_odds', 'over_ticket_pct', 'over_money_pct',
        'under_ticket_pct', 'under_money_pct', 'home_spread', 'home_spread_odds',
        'home_spread_ticket_pct', 'home_spread_money_pct', 'away_spread',
        'away_spread_odds', 'away_spread_ticket_pct', 'away_spread_money_pct'
    ]

    next_games_list = df['game_id'].unique().tolist()

    display(filtered_df.sample(4))

    games_list = filtered_df.loc[filtered_df['game_id'].isin(next_games_list)].groupby(['game_id','home_team','away_team','start_time_pt']).agg(
        rec_count=('date_scraped','size'),
        num_bets=('num_bets','last')
    
    
    
    ).sort_values('num_bets', ascending=False).head(30).reset_index()['game_id'].tolist()


    for col in metric_columns:
        if col in filtered_df.columns:
            # This converts the column to a numeric type.
            # Any string like 'N/A' will become NaN.
            filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

    df_agg = aggregate_betting_data(filtered_df.loc[filtered_df['game_id'].isin(games_list)], group_by_columns, metric_columns)

    df_agg = df_agg.sort_values('start_time',ascending=True)

    # Create the home_team_spread column
    df_agg['home_team_spread'] = df_agg['home_team'] + " " + df_agg['home_spread_last'].apply(lambda x: f"{x:+.1f}")

    # Create the away_team_spread column (assuming this is the second column you wanted)
    df_agg['away_team_spread'] = df_agg['away_team'] + " " + df_agg['away_spread_last'].apply(lambda x: f"{x:+.1f}")

    # display(df_agg[['home_team','away_team','home_spread_first','home_spread_last','home_team_spread','away_team_spread']])

    # display(df_agg[['home_team','away_team','home_spread_first','home_spread_last','home_spread_ticket_pct_first','home_spread_ticket_pct_last']])


    df_hist = pd.read_csv('../data/ncaab_bet_picks_evaluated.csv')
    df_hist = df_hist.loc[df_hist['model'] == model_version]

    # df_hist = pd.DataFrame()

    df1_string = df_agg.to_csv(index=False)
    df2_string = df_hist.to_csv(index=False)

    timestamp_str = datetime.datetime.now()

    prompt = f"""
    You are my expert college basketball betting adviser.
    I will provide you with two datasets:

    Dataset 1: Betting lines for upcoming games (money line, over/under, spread with first/avg/last values)
    Dataset 2: Historical betting results to analyze what's working and what's not

    Your goal: Maximize ROI by learning from historical patterns.

    CRITICAL VALIDATION REQUIREMENTS
    1. HOME vs AWAY TEAM IDENTIFICATION - READ CAREFULLY
    The dataset has two columns: home_team and away_team
    MATCH NAMING CONVENTION (MANDATORY):

    ALWAYS use format: "home_team vs away_team"
    Example: If home_team=Thunder, away_team=Wizards → Write "Thunder vs Wizards"
    The home team is ALWAYS listed first, away team second
    This makes it crystal clear which team is playing at home

    BEFORE MAKING ANY PICK:

    Identify from the dataset: Which team is in the home_team column?
    Identify from the dataset: Which team is in the away_team column?
    Write the match as "home_team vs away_team"
    Determine which team you want to pick
    Set the binary indicator based on whether that team is home or away

    BINARY INDICATOR RULES:

    If you pick the HOME team's spread → bet_home_spread=1, bet_away_spread=0
    If you pick the AWAY team's spread → bet_away_spread=1, bet_home_spread=0
    If you pick the HOME team's ML → bet_home_ml=1, bet_away_ml=0
    If you pick the AWAY team's ML → bet_away_ml=1, bet_home_ml=0

    EXAMPLE:
    Dataset shows: home_team=Thunder, away_team=Wizards
    Match name: "Thunder vs Wizards"
    If picking Thunder -15.5: bet_home_spread=1 (Thunder is home)
    If picking Wizards +15.5: bet_away_spread=1 (Wizards is away)

    ---
    ### **2. ODDS AND LINES VALIDATION - NO EXCEPTIONS**

    **Use ONLY the "_last" column values:**
    - `home_money_line_last` for home team ML
    - `away_money_line_last` for away team ML
    - `home_spread_last` and `home_spread_odds_last` for home team spread
    - `away_spread_last` and `away_spread_odds_last` for away team spread
    - `total_score_last`, `over_odds_last`, `under_odds_last` for totals

    **NEVER:**
    - Invent odds
    - Approximate odds
    - Use "avg" or "first" values (only use for analysis of line movement)
    - Make a pick if the line is not in the dataset

    ---

    ### **3. SPREAD DIRECTION RULES - READ CAREFULLY**

    **Understanding Spread Signs:**
    - **NEGATIVE spread (-X.X)** = That team is FAVORED by X.X points
    - **POSITIVE spread (+X.X)** = That team is UNDERDOG getting X.X points

    **Examples:**
    - `home_spread_last = -5.5` means: Home team FAVORED by 5.5, Away team gets +5.5
    - `home_spread_last = +3.5` means: Home team UNDERDOG getting +3.5, Away team favored by -3.5
    - `away_spread_last = -7.0` means: Away team FAVORED by 7.0, Home team gets +7.0
    - `away_spread_last = +4.0` means: Away team UNDERDOG getting +4.0, Home team favored by -4.0

    **Critical Understanding:**
    - If `home_spread_last` is negative → home team is favorite
    - If `home_spread_last` is positive → home team is underdog
    - If `away_spread_last` is negative → away team is favorite
    - If `away_spread_last` is positive → away team is underdog

    ---

    ### **4. MANDATORY DOUBLE-CHECK PROCESS**

    **Before finalizing EACH pick, complete these steps:**

    □ **Step 1**: Look at dataset - which team is `home_team`, which is `away_team`?
    □ **Step 2**: Write match as "home_team vs away_team"
    □ **Step 3**: Decide which team I want to pick
    □ **Step 4**: Is that team home or away?
    □ **Step 5**: Look up the EXACT line for that team in the "_last" columns
    □ **Step 6**: Copy the EXACT odds from the corresponding "_odds_last" column
    □ **Step 7**: Verify the sign (+ or -) matches favorite/underdog position
    □ **Step 8**: Set binary indicator: bet_home_X=1 if home team, bet_away_X=1 if away team
    □ **Step 9**: Cross-check one final time before writing

    **If you are uncertain about ANY detail, SKIP THAT PICK rather than guess.**

    ---

    ### **5. PICK TYPES AND BINARY INDICATORS**

    Please aim to make around 10 picks -- you can pick more or less, but I want to have at least 10 and then we can use the confidence to determine success

    You can make six types of picks:

    | Pick Type | Columns to Use | Binary Indicators |
    |-----------|----------------|-------------------|
    | Home ML | `home_money_line_last` | `bet_home_ml=1, bet_away_ml=0` |
    | Away ML | `away_money_line_last` | `bet_away_ml=1, bet_home_ml=0` |
    | Home Spread | `home_spread_last`, `home_spread_odds_last` | `bet_home_spread=1, bet_away_spread=0` |
    | Away Spread | `away_spread_last`, `away_spread_odds_last` | `bet_away_spread=1, bet_home_spread=0` |
    | Over | `total_score_last`, `over_odds_last` | `bet_over=1, bet_under=0` |
    | Under | `total_score_last`, `under_odds_last` | `bet_under=1, bet_over=0` |

    **All other binary indicators must be set to 0.**

    ---

    ### **6. CONFIDENCE & UNITS**

    - Rank all picks by confidence (most confident = rank 1)
    - Provide **confidence %** as integer between 0-100
    - Assign units based on confidence:
    - **3 units**: Highest confidence (90%+)
    - **2 units**: Medium confidence (80-89%)
    - **1 unit**: Lower confidence (70-79%)

    ---

    ### **7. PREDICTED SCORE FORMAT**

    - Format: "HomeScore-AwayScore" (e.g., "115-112")
    - Home team score ALWAYS listed first
    - Away team score ALWAYS listed second
    - Double-check the order matches your match naming

    ---

    ## **OUTPUT FORMAT**

    ### **Part 1: Human-Readable Table**

    Create a table with these columns:
    - Rank
    - Match (format: "home_team vs away_team")
    - Home Team
    - Away Team
    - Pick (e.g., "Thunder -15.5" or "Wizards +15.5")
    - Odds
    - Units
    - Confidence %
    - Reason - a well thought out reason why you are making the pick that you are
    - Reason Code - a codified reason for the decision. I will use these to track patterns and repeat successful strategies. This should be a shorter form code that is reused across multiple datasets.
    - Predicted Score (format: "HomeScore-AwayScore")

    ### **Part 2: CSV Block (Copy/Paste Ready)**

    Exact structure with this header row:
    ```
    rank,game_id,start_time,match,pick,odds,units,confidence_pct,reason,predicted_score,bet_home_spread,bet_home_ml,bet_away_spread,bet_away_ml,bet_over,bet_under,home_money_line,away_money_line,tie_money_line,total_score,over_odds,under_odds,home_spread,home_spread_odds,away_spread,away_spread_odds,timestamp
    ```
    **CSV Requirements:**
    - `match`: Must use "home_team vs away_team" format
    - `home_team`: home team
    - `away_team`: away team
    - `pick`: State team name and line (e.g., "Thunder -15.5")
    - `predicted_score`: Format as "HomeScore-AwayScore"
    - `bet_home_spread`, `bet_home_ml`, `bet_away_spread`, `bet_away_ml`, `bet_over`, `bet_under`: Must be 0 or 1
    - `home_money_line`: Value from `home_money_line_last`
    - `away_money_line`: Value from `away_money_line_last`
    - `tie_money_line`: Always "N/A"
    - `total_score`: Value from `total_score_last`
    - `over_odds`: Value from `over_odds_last`
    - `under_odds`: Value from `under_odds_last`
    - `home_spread`: Value from `home_spread_last`
    - `home_spread_odds`: Value from `home_spread_odds_last`
    - `away_spread`: Value from `away_spread_last`
    - `away_spread_odds`: Value from `away_spread_odds_last`
    - `timestamp`: use the time of this prompt -- {timestamp_str}

    ---

    ## **FINAL VERIFICATION CHECKLIST**

    Before submitting your picks, verify:

    □ Every match uses "home_team vs away_team" format
    □ Every pick references the correct team (home or away)
    □ Every odds value is copied exactly from "_last" column
    □ Every binary indicator correctly reflects whether the picked team is home or away
    □ Every spread sign (+ or -) matches the favorite/underdog position
    □ Every predicted score is in "HomeScore-AwayScore" format
    □ All CSV columns match the exact structure required

    ---

    ## **EXAMPLE OF CORRECT PICK**

    **Dataset shows:**
    - game_id: 261702
    - home_team: Thunder
    - away_team: Wizards
    - home_spread_last: -15.5
    - home_spread_odds_last: -110

    **Correct Pick:**
    - Match: "Thunder vs Wizards"
    - Pick: "Thunder -15.5"
    - Odds: -110
    - Binary: bet_home_spread=1, bet_away_spread=0, all others=0
    - Predicted Score: "126-108" (Thunder score first)

    **CSV Line:**
    ```
    1,261702,2025-10-31T00:00:00.000Z,Thunder vs Wizards,Thunder -15.5,-110,3,96,"Reason here",126-108,1,0,0,0,0,0,-1200,750,N/A,231.5,-110,-109,-15.5,-110,15.5,-110,2025-10-30T18:30:00Z

    Remember: Accuracy is more important than quantity. Skip any pick where you have uncertainty.


    Here are the upcoming games and their odds:
    {df1_string}

    Here is the historical dataset of your betting advice and results:
    {df2_string}
        """
    print('-------')
    print('-------')
    print('-------')
    print('-------')

    # Now you can print the full prompt
    print(prompt)

    ## write prompt to a text file
    with open(f"../prompts/ncaab_prompt_{model_version}.txt", "w") as f:
        f.write(prompt) 

    return df_agg



In [34]:
HEADERS = {
    'Authority': 'api.actionnetwork',
    'Accept': 'application/json',
    'Origin': 'https://www.actionnetwork.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}

def process_results(model_name: str, picks_dir: Path, results_csv_path: Path):
    """
    Processes betting picks for a given model against a game results CSV.
    
    It finds missing game results, fetches them from an API, appends them
    to the main results CSV, and then evaluates all picks.
    """
    sport = 'ncaab'
    
    # === 1. Load Picks File ===
    picks_file = picks_dir / f'cbb_bets_{model_name}.txt'
    try:
        df_picks = pd.read_csv(picks_file)
    except FileNotFoundError:
        print(f"Error: Picks file not found at {picks_file}")
        return None # Exit function
    
    display(df_picks.sample(3))

    # === 2. Process Pick Timestamps ===
    df_picks['timestamp'] = pd.to_datetime(df_picks['timestamp'], format='ISO8601')
    df_picks['start_time_pt'] = (
        pd.to_datetime(df_picks['start_time'], utc=True)
        .dt.tz_convert('America/Los_Angeles')
    )
    df_picks['date'] = df_picks['start_time_pt'].dt.date
    df_picks['model'] = model_name

    # === 3. Load Existing Game Results ===
    # Check if the file exists *before* trying to read it.
    # This is key for knowing whether to write the header later.
    results_file_exists = results_csv_path.is_file()
    
    try:
        df_old_results = pd.read_csv(results_csv_path)
    except FileNotFoundError:
        print(f"Results file {results_csv_path} not found. A new one will be created.")
        df_old_results = pd.DataFrame() # Start with an empty DataFrame

    # === 4. Find Missing Results ===
    df_merge = pd.merge(
        df_picks[['rank', 'game_id', 'match', 'date', 'start_time', 'pick']],
        df_old_results,
        on='game_id',
        how='left',
        suffixes=('_pick', '_result')
    )
    missing_games = df_merge.loc[df_merge['status'] != 'complete']

    # === 5. Fetch and Append New Results (if any) ===
    df_new_results = pd.DataFrame() # Initialize as empty

    if not missing_games.empty:
        date_str_list = missing_games['date'].astype(str).str.replace('-', '').unique().tolist()
        
        if date_str_list:
            print(f"Found missing results for {len(date_str_list)} dates. Fetching...")
            df_new_results = get_complete_game_results(sport, date_str_list, HEADERS)
            
            if not df_new_results.empty:
                print(f"Appending {len(df_new_results)} new results to {results_csv_path}")
                # Append new data
                df_new_results.to_csv(
                    results_csv_path,
                    mode='a',
                    # Write header ONLY if the file didn't exist before
                    header=not results_file_exists, 
                    index=False
                )
            else:
                print("API call returned no new results.")
    else:
        print("No missing game results found. All picks are up-to-date.")

    # === 6. Combine All Results for Final Processing ===
    # *** THIS IS THE MAIN LOGIC FIX ***
    # Combine the old results and the brand-newly fetched results
    df_all_results = pd.concat([df_old_results, df_new_results], ignore_index=True)

    # Drop duplicates in case the API sent a game we already had
    if 'game_id' in df_all_results.columns and not df_all_results.empty:
        df_all_results = df_all_results.drop_duplicates(subset='game_id', keep='last')

    # === 7. Process and Save Evaluations ===
    # Pass the *complete* set of results (old + new)
    df_evaluated, df_evaluated_hist = process_and_save_evaluated_bets(
        df_picks, 
        df_all_results, 
        sport
    )

    if 'date' in df_evaluated_hist.columns:
        df_evaluated_hist['date'] = pd.to_datetime(df_evaluated_hist['date'])
    else:
        print("Warning: 'df_evaluated_hist' has no 'date' column to convert.")

    return df_evaluated_hist

# Load Hist

In [56]:
df_hist = pd.read_csv('../data/ncaab_bet_picks.csv')
model_list = df_hist['model'].unique().tolist()
print(model_list)
display(df_hist.sample(3))

['perp', 'claude']


,rank,game_id,start_time,match,pick,odds,units,confidence_pct,reason,predicted_score,...,over_odds,under_odds,home_spread,home_spread_odds,away_spread,away_spread_odds,timestamp,start_time_pt,date,model
76,11,264790,2025-11-15T02:00:00.000Z,TCU vs Michigan,TCU +7.0,-110,1,70,Home dog getting a full touchdown. Michigan 99...,73-78,...,-110,-110,7.0,-110,-7.0,-110,2025-11-13 15:00:25.271884,2025-11-14 18:00:00-08:00,2025-11-14,claude
3,4,264732,2025-11-12T01:00:00.000Z,Louisville vs Kentucky,Kentucky +6.5,-112,1,78,Rivalry with live road dog under a TD; fading ...,84-79,...,-115,-108,-6.5,-108,6.5,-112,2025-11-11 09:53:25.723285,2025-11-11 17:00:00-08:00,2025-11-11,perp
23,4,266373,2025-11-13T00:00:00.000Z,Marshall vs Elon,Marshall -9.5,-105,2,82,Single-digit home chalk with tempo edge; below...,84-72,...,-110,-110,-9.5,-105,9.5,-115,2025-11-12 12:46:41.533925,2025-11-12 16:00:00-08:00,2025-11-12,perp


# Build Prompts

In [63]:

pd.set_option('display.max_colwidth', 500)
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)

In [68]:
df.sort_values('num_bets_last',ascending=False).head(10)

,game_id,home_team,away_team,start_time,num_bets_first,num_bets_avg,num_bets_last,home_money_line_first,home_money_line_avg,home_money_line_last,home_ml_ticket_pct_first,home_ml_ticket_pct_avg,home_ml_ticket_pct_last,home_ml_money_pct_first,home_ml_money_pct_avg,home_ml_money_pct_last,away_money_line_first,away_money_line_avg,away_money_line_last,away_ml_ticket_pct_first,away_ml_ticket_pct_avg,away_ml_ticket_pct_last,away_ml_money_pct_first,away_ml_money_pct_avg,away_ml_money_pct_last,total_score_first,total_score_avg,total_score_last,over_odds_first,over_odds_avg,over_odds_last,under_odds_first,under_odds_avg,under_odds_last,over_ticket_pct_first,over_ticket_pct_avg,over_ticket_pct_last,over_money_pct_first,over_money_pct_avg,over_money_pct_last,under_ticket_pct_first,under_ticket_pct_avg,under_ticket_pct_last,under_money_pct_first,under_money_pct_avg,under_money_pct_last,home_spread_first,home_spread_avg,home_spread_last,home_spread_odds_first,home_spread_odds_avg,home_spread_odds_last,home_spread_ticket_pct_first,home_spread_ticket_pct_avg,home_spread_ticket_pct_last,home_spread_money_pct_first,home_spread_money_pct_avg,home_spread_money_pct_last,away_spread_first,away_spread_avg,away_spread_last,away_spread_odds_first,away_spread_odds_avg,away_spread_odds_last,away_spread_ticket_pct_first,away_spread_ticket_pct_avg,away_spread_ticket_pct_last,away_spread_money_pct_first,away_spread_money_pct_avg,away_spread_money_pct_last,home_team_spread,away_team_spread
7,264807,UConn,BYU,2025-11-16T00:00:00.000Z,9203.0,6960.25,9203.0,-294.0,-265.5,-294.0,92.0,69.0,92.0,82.0,61.5,82.0,237,215.25,237,8,6.0,8,18,13.5,18,153.5,153.75,153.5,-109,-109.25,-109,-112,-111.5,-112,30,22.5,30,33,24.75,33,70,52.5,70,67,50.25,67,-6.5,-5.75,-6.5,-105,-106.25,-105,60,45.0,60,55,41.25,55,6.5,5.75,6.5,-115,-113.75,-115,40,30.0,40,45,33.75,45,UConn -6.5,BYU +6.5
16,264823,Marquette,Maryland,2025-11-15T19:00:00.000Z,4998.0,4998.00,4998.0,-370.0,-370.0,-370.0,98.0,98.0,98.0,95.0,95.0,95.0,290,290.00,290,2,2.0,2,5,5.0,5,145.5,145.50,145.5,-115,-115.00,-115,-111,-111.0,-111,74,74.0,74,73,73.00,73,26,26.0,26,27,27.00,27,-8.5,-8.50,-8.5,-110,-110.00,-110,59,59.0,59,55,55.00,55,8.5,8.50,8.5,-110,-110.00,-110,41,41.0,41,45,45.00,45,Marquette -8.5,Maryland +8.5
4,264802,SMU,Butler,2025-11-15T19:00:00.000Z,3983.0,3983.00,3983.0,-360.0,-360.0,-360.0,97.0,97.0,97.0,93.0,93.0,93.0,270,270.00,270,3,3.0,3,7,7.0,7,164.5,164.50,164.5,-105,-105.00,-105,-110,-110.0,-110,12,12.0,12,13,13.00,13,88,88.0,88,87,87.00,87,-7.5,-7.50,-7.5,-110,-110.00,-110,46,46.0,46,39,39.00,39,7.5,7.50,7.5,-110,-110.00,-110,54,54.0,54,61,61.00,61,SMU -7.5,Butler +7.5
14,264820,Nebraska,Oklahoma,2025-11-16T00:00:00.000Z,3322.0,3322.00,3322.0,-112.0,-112.0,-112.0,80.0,80.0,80.0,82.0,82.0,82.0,-108,-108.00,-108,20,20.0,20,18,18.0,18,163.5,163.50,163.5,-105,-105.00,-105,-115,-115.0,-115,7,7.0,7,8,8.00,8,93,93.0,93,92,92.00,92,-1.5,-1.50,-1.5,-110,-110.00,-110,48,48.0,48,19,19.00,19,1.5,1.50,1.5,-111,-111.00,-111,52,52.0,52,81,81.00,81,Nebraska -1.5,Oklahoma +1.5
24,267559,Temple,Boston Col,2025-11-15T19:00:00.000Z,3145.0,3145.00,3145.0,-173.0,-173.0,-173.0,98.0,98.0,98.0,98.0,98.0,98.0,145,145.00,145,2,2.0,2,2,2.0,2,141.5,141.50,141.5,-112,-112.00,-112,-108,-109.0,-110,71,71.0,71,72,72.00,72,29,29.0,29,28,28.00,28,-3.5,-3.50,-3.5,-110,-110.00,-110,67,67.0,67,71,71.00,71,3.5,3.50,3.5,-110,-110.00,-110,33,33.0,33,29,29.00,29,Temple -3.5,Boston Col +3.5
17,264825,Denver,UTSA,2025-11-15T19:00:00.000Z,2367.0,2367.00,2367.0,-205.0,-205.0,-205.0,95.0,95.0,95.0,91.0,91.0,91.0,164,164.00,164,5,5.0,5,9,9.0,9,148.5,148.50,148.5,-105,-105.00,-105,-115,-115.0,-115,34,34.0,34,33,33.00,33,66,66.0,66,67,67.00,67,-4.5,-4.50,-4.5,-108,-108.00,-108,62,62.0,62,69,69.00,69,4.5,4.50,4.5,-112,-112.00,-112,38,38.0,38,31,31.00,31,Denver -4.5,UTSA +4.5
21,266396,Kent State,Cleveland St,2025-11-15T18:00:00.000Z,1984.0,1984.00,1984.0,-650.0,-650.0,-650.0,98.0,98.0,98.0,93.0,93.0,93.0,470,470.00,470,2,2.0,2,7,7.0,7,165.5,165.50,

In [83]:
model_list

['perp', 'claude', 'gemini']

In [82]:
for model_name in model_list:
    df = build_ncaa_prompt(model_name)

Processing data for ncaab (date: 20251118)
global version
Fetching data from the Action Network API...
https://api.actionnetwork.com/web/v2/scoreboard/ncaab?bookIds=15,30,79,2988,75,123,71,68,69&periods=event&date=20251118&division=D1
Data successfully fetched.

--- Game Details ---
Game ID: 266418
League: ncaab
Matchup: Coppin St vs Knights
Home Team ID: 906
Away Team ID: 1706
Status: scheduled
Start Time: 2025-11-18T16:00:00.000Z
Home Score: N/A
Away Score: N/A
--------------------
Game ID: 266418, Market ID: 15 - No moneyline data available. Skipping...
Game ID: 266418, Market ID: 30 - No moneyline data available. Skipping...
Game ID: 266418 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 264866
League: ncaab
Matchup: Kentucky vs Michigan St
Home Team ID: 979
Away Team ID: 1009
Status: scheduled
Start Time: 2025-11-18T23:30:00.000Z
Home Score: N/A
Away Score: N/A
--------------------
Processing Game ID: 264866, Market ID: 15
Processi

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Data successfully fetched.

--- Game Details ---
Game ID: 266454
League: ncaab
Matchup: Yale vs Green Bay
Home Team ID: 1039
Away Team ID: 958
Status: scheduled
Start Time: 2025-11-21T17:30:00.000Z
Home Score: N/A
Away Score: N/A
--------------------
Game ID: 266454 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 266455
League: ncaab
Matchup: Tulane vs Utah State
Home Team ID: 820
Away Team ID: 971
Status: scheduled
Start Time: 2025-11-21T18:00:00.000Z
Home Score: N/A
Away Score: N/A
--------------------
Game ID: 266455 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 264526
League: ncaab
Matchup: IUIN vs Alabama St
Home Team ID: 842
Away Team ID: 892
Status: scheduled
Start Time: 2025-11-21T19:00:00.000Z
Home Score: N/A
Away Score: N/A
--------------------
Game ID: 264526 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 264935
League: ncaa

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/t7/mm5m89410_3fmyq9v_19k2dm0000gn/T/ipykernel_57841/1220778606.py:51: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



,game_id,league_name,home_team,away_team,home_team_id,away_team_id,status,home_score,away_score,home_pitcher,home_pitcher_era,home_pitcher_k9,home_pitcher_ip,home_pitcher_starts,home_pitcher_win,home_pitcher_loss,away_pitcher,away_pitcher_era,away_pitcher_k9,away_pitcher_ip,away_pitcher_starts,away_pitcher_win,away_pitcher_loss,start_time,market_id,book_id,event_id,num_bets,home_money_line,home_ml_ticket_pct,home_ml_money_pct,away_money_line,away_ml_ticket_pct,away_ml_money_pct,tie_money_line,tie_ml_ticket_pct,tie_ml_money_pct,total_score,over_odds,under_odds,over_ticket_pct,over_money_pct,under_ticket_pct,under_money_pct,home_spread,home_spread_odds,home_spread_ticket_pct,home_spread_money_pct,away_spread,away_spread_odds,away_spread_ticket_pct,away_spread_money_pct,start_time_pt,date_scraped
425,266386,ncaab,Georgia,G Tech,980,913,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T02:00:00.000Z,15,15.0,266386.0,1923.0,-1600.0,96.0,98.0,900,4,2,NaN,NaN,NaN,150.5,-115,-111,67,69,33,31,-15.5,-105,67,88,15.5,-115,33,12,2025-11-14 18:00:00-08:00,2025-11-14 07:58:57.845341
8,264870,ncaab,Buffalo,Vermont,1001,1018,scheduled,N/A,N/A,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T23:30:00.000Z,15,15,264870,1317.0,130,32,10,-155,68,90,N/A,N/A,N/A,140.5,-113,-107,82,81,18,19,2.5,-105,39,23,-2.5,-115,61,77,2025-11-18 15:30:00-08:00,2025-11-18 07:15:57.768562
327,266403,ncaab,CS Fullerton,Pacific,852,1062,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T22:00:00.000Z,15,15.0,266403.0,656.0,230.0,4.0,11.0,-278,96,89,NaN,NaN,NaN,160.5,-110,-110,21,26,79,74,6.5,-111,0,0,-6.5,-111,0,0,2025-11-15 14:00:00-08:00,2025-11-15 10:09:29.999661
66,267629,ncaab,UCF,Oakland,816,961,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-18T00:00:00.000Z,15,15.0,267629.0,5482.0,-1100.0,91.0,98.0,700,9,2,NaN,NaN,NaN,162.5,-115,-111,47,48,53,52,-13.5,-110,44,62,13.5,-110,56,38,2025-11-17 16:00:00-08:00,2025-11-17 14:55:34.575846


...

,game_id,league_name,home_team,away_team,home_team_id,away_team_id,status,home_score,away_score,home_pitcher,home_pitcher_era,home_pitcher_k9,home_pitcher_ip,home_pitcher_starts,home_pitcher_win,home_pitcher_loss,away_pitcher,away_pitcher_era,away_pitcher_k9,away_pitcher_ip,away_pitcher_starts,away_pitcher_win,away_pitcher_loss,start_time,market_id,book_id,event_id,num_bets,home_money_line,home_ml_ticket_pct,home_ml_money_pct,away_money_line,away_ml_ticket_pct,away_ml_money_pct,tie_money_line,tie_ml_ticket_pct,tie_ml_money_pct,total_score,over_odds,under_odds,over_ticket_pct,over_money_pct,under_ticket_pct,under_money_pct,home_spread,home_spread_odds,home_spread_ticket_pct,home_spread_money_pct,away_spread,away_spread_odds,away_spread_ticket_pct,away_spread_money_pct,start_time_pt,date_scraped
189,267343,ncaab,Jax State,Coastal Car,760,796,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T00:00:00.000Z,15,15.0,267343.0,706.0,-312.0,98.0,98.0,260.0,2.0,2.0,NaN,NaN,NaN,138.5,-108,-112,78,76,22,24,-7.5,-105,75,88,7.5,-115.0,25.0,12.0,2025-11-14 16:00:00-08:00,2025-11-14 07:10:41.949757
323,264757,ncaab,Missouri,Minnesota,983,1016,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-13T01:00:00.000Z,15,15.0,264757.0,3700.0,-310.0,97.0,73.0,250.0,3.0,27.0,NaN,NaN,NaN,156.5,-116,-110,27,27,73,73,-6.5,-110,57,67,6.5,-111.0,43.0,33.0,2025-11-12 17:00:00-08:00,2025-11-12 09:35:53.735006
32,264778,ncaab,Lindenwood,Charleston So,1477,951,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-14T23:30:00.000Z,15,15.0,264778.0,1055.0,-142.0,14.0,3.0,120.0,86.0,97.0,NaN,NaN,NaN,156.5,-114,-110,53,51,47,49,-3.5,-113,34,66,3.5,-106.0,66.0,34.0,2025-11-14 15:30:00-08:00,2025-11-14 07:58:57.845341
408,264783,ncaab,Colorado,Providence,755,1067,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T02:00:00.000Z,15,15.0,264783.0,1620.0,-108.0,43.0,2.0,-111.0,57.0,98.0,NaN,NaN,NaN,163.5,-108,-112,48,46,52,54,1.5,-120,55,76,-1.5,-102.0,45.0,24.0,2025-11-14 18:00:00-08:00,2025-11-14 07:10:41.084955


-------
-------
-------
-------

    You are my expert college basketball betting adviser.
    I will provide you with two datasets:

    Dataset 1: Betting lines for upcoming games (money line, over/under, spread with first/avg/last values)
    Dataset 2: Historical betting results to analyze what's working and what's not

    Your goal: Maximize ROI by learning from historical patterns.

    CRITICAL VALIDATION REQUIREMENTS
    1. HOME vs AWAY TEAM IDENTIFICATION - READ CAREFULLY
    The dataset has two columns: home_team and away_team
    MATCH NAMING CONVENTION (MANDATORY):

    ALWAYS use format: "home_team vs away_team"
    Example: If home_team=Thunder, away_team=Wizards → Write "Thunder vs Wizards"
    The home team is ALWAYS listed first, away team second
    This makes it crystal clear which team is playing at home

    BEFORE MAKING ANY PICK:

    Identify from the dataset: Which team is in the home_team column?
    Identify from the dataset: Which team is in the away_team c

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/aaronsmith/Code/Void/First Void App/llm_betting_model/scripts/utils.py:512: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or a

,game_id,league_name,home_team,away_team,home_team_id,away_team_id,status,home_score,away_score,home_pitcher,home_pitcher_era,home_pitcher_k9,home_pitcher_ip,home_pitcher_starts,home_pitcher_win,home_pitcher_loss,away_pitcher,away_pitcher_era,away_pitcher_k9,away_pitcher_ip,away_pitcher_starts,away_pitcher_win,away_pitcher_loss,start_time,market_id,book_id,event_id,num_bets,home_money_line,home_ml_ticket_pct,home_ml_money_pct,away_money_line,away_ml_ticket_pct,away_ml_money_pct,tie_money_line,tie_ml_ticket_pct,tie_ml_money_pct,total_score,over_odds,under_odds,over_ticket_pct,over_money_pct,under_ticket_pct,under_money_pct,home_spread,home_spread_odds,home_spread_ticket_pct,home_spread_money_pct,away_spread,away_spread_odds,away_spread_ticket_pct,away_spread_money_pct,start_time_pt,date_scraped
186,264792,ncaab,Wolves,Citadel,1405,828,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T00:00:00.000Z,15,15.0,264792.0,1807.0,-298.0,96.0,97.0,240.0,4.0,3.0,NaN,NaN,NaN,150.0,-107,-112,57,59,43,41,-6.5,-114,76,89,6.5,-105.0,24.0,11.0,2025-11-14 16:00:00-08:00,2025-11-14 10:45:56.182855
423,266380,ncaab,Texas St,UTSA,795,864,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-13T01:00:00.000Z,15,15.0,266380.0,1201.0,-155.0,83.0,84.0,130.0,17.0,16.0,NaN,NaN,NaN,147.5,-110,-110,47,45,53,55,-3.5,-105,67,44,3.5,-115.0,33.0,56.0,2025-11-12 17:00:00-08:00,2025-11-12 12:46:40.502649
583,264782,ncaab,Washington St,Washington,750,758,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-15T04:00:00.000Z,15,15.0,264782.0,1211.0,320.0,4.0,3.0,-410.0,96.0,97.0,NaN,NaN,NaN,158.5,-110,-111,52,52,48,48,8.5,-110,47,28,-8.5,-110.0,53.0,72.0,2025-11-14 20:00:00-08:00,2025-11-14 10:12:39.958320
159,264857,ncaab,Colorado,Alabama St,755,892,scheduled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-11-18T02:00:00.000Z,15,15.0,264857.0,1751.0,-8000.0,83.0,92.0,2200.0,17.0,8.0,NaN,NaN,NaN,156.5,-107,-109,58,60,42,40,-22.5,-110,19,30,22.5,-110.0,81.0,70.0,2025-11-17 18:00:00-08:00,2025-11-17 14:55:33.783408


-------
-------
-------
-------

    You are my expert college basketball betting adviser.
    I will provide you with two datasets:

    Dataset 1: Betting lines for upcoming games (money line, over/under, spread with first/avg/last values)
    Dataset 2: Historical betting results to analyze what's working and what's not

    Your goal: Maximize ROI by learning from historical patterns.

    CRITICAL VALIDATION REQUIREMENTS
    1. HOME vs AWAY TEAM IDENTIFICATION - READ CAREFULLY
    The dataset has two columns: home_team and away_team
    MATCH NAMING CONVENTION (MANDATORY):

    ALWAYS use format: "home_team vs away_team"
    Example: If home_team=Thunder, away_team=Wizards → Write "Thunder vs Wizards"
    The home team is ALWAYS listed first, away team second
    This makes it crystal clear which team is playing at home

    BEFORE MAKING ANY PICK:

    Identify from the dataset: Which team is in the home_team column?
    Identify from the dataset: Which team is in the away_team c

# Evaluate Results

In [76]:
model_list

['perp', 'claude']

In [77]:
model_list = ['perp','claude','gemini']

In [78]:

for model_name in model_list:
    base_dir = Path('../data')
    results_file = Path('../data/ncaab_game_results.csv')
    df_evaluated_hist = process_results(model_name, base_dir, results_file)

,rank,game_id,start_time,match,pick,odds,units,confidence_pct,reason,predicted_score,bet_home_spread,bet_home_ml,bet_away_spread,bet_away_ml,bet_over,bet_under,home_money_line,away_money_line,tie_money_line,total_score,over_odds,under_odds,home_spread,home_spread_odds,away_spread,away_spread_odds,timestamp
61,5,264812,2025-11-15T21:00:00.000Z,St. Bonaventure vs Youngstown St,St. Bonaventure -10.5,-110,2,82,Double-digit but not extreme; Bona’s home edge plus non-conference dynamic supports laying 10.5 [web:25][web:6][web:13].,77-64,1,0,0,0,0,0,-587.0,445,NaN,148.5,-105,-110,-10.5,-110,10.5,-110,2025-11-15 10:15:03.538272
1,2,264701,2025-11-12T01:30:00.000Z,Illinois vs Texas Tech,Illinois -3.5,-115,2,84,"Short home favorite within typical non-conference HCA band (~3–3.5), avoiding big-chalk volatility; edge to modest home favorite over coin-flip public splits [web:13][web:6][web:25].",86-80,1,0,0,0,0,0,-180.0,147,NaN,167.5,-115,-110,-3.5,-115,3.5,-105,2025-11-11 09:53:25.723285
39,7,264766,2025-11-14T02:00:00.000Z,K State vs California,K State -6.5,-112,1,79,Short home fav mirrors HCA without overpaying; number held 6.5 on “last” with supportive home profile [web:13][web:6][web:15].,80-71,1,0,0,0,0,0,-287.0,232,NaN,163.5,-115,-108,-6.5,-112,6.5,-108,2025-11-13 15:00:26.036721


Found missing results for 1 dates. Fetching...
Processing data for ncaab (date: 20251117)
global version
Fetching data from the Action Network API...
https://api.actionnetwork.com/web/v2/scoreboard/ncaab?bookIds=15,30,79,2988,75,123,71,68,69&periods=event&date=20251117&division=D1
Data successfully fetched.

--- Game Details ---
Game ID: 266416
League: ncaab
Matchup: Georgia vs Florida A&M
Home Team ID: 980
Away Team ID: 909
Status: complete
Start Time: 2025-11-17T23:30:00.000Z
Home Score: 87
Away Score: 57
--------------------
Game ID: 266416 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 264848
League: ncaab
Matchup: Pittsburgh vs Bucknell
Home Team ID: 914
Away Team ID: 944
Status: complete
Start Time: 2025-11-18T00:00:00.000Z
Home Score: 84
Away Score: 50
--------------------
Game ID: 264848 - No valid markets found. Appending game data without market info.

--- Game Details ---
Game ID: 264849
League: ncaab
Matchup: NC State vs VC

,game_id,league_name,home_team,away_team,home_team_id,away_team_id,status,home_score,away_score,home_pitcher,home_pitcher_era,home_pitcher_k9,home_pitcher_ip,home_pitcher_starts,home_pitcher_win,home_pitcher_loss,away_pitcher,away_pitcher_era,away_pitcher_k9,away_pitcher_ip,away_pitcher_starts,away_pitcher_win,away_pitcher_loss,start_time,market_id,book_id,event_id,num_bets,home_money_line,home_ml_ticket_pct,home_ml_money_pct,away_money_line,away_ml_ticket_pct,away_ml_money_pct,tie_money_line,tie_ml_ticket_pct,tie_ml_money_pct,total_score,over_odds,under_odds,over_ticket_pct,over_money_pct,under_ticket_pct,under_money_pct,home_spread,home_spread_odds,home_spread_ticket_pct,home_spread_money_pct,away_spread,away_spread_odds,away_spread_ticket_pct,away_spread_money_pct,start_time_pt
0,266416,ncaab,Georgia,Florida A&M,980,909,complete,87,57,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-17T23:30:00.000Z,N/A,N/A,N/A,3549.0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2025-11-17 15:30:00-08:00
1,264848,ncaab,Pittsburgh,Bucknell,914,944,complete,84,50,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,N/A,N/A,N/A,3084.0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2025-11-17 16:00:00-08:00
2,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,15,15,264849,10264.0,-900,93,85,577,7,15,N/A,N/A,N/A,164.5,-112,-110,62,63,38,37,-11.5,-115,54,70,N/A,N/A,N/A,N/A,2025-11-17 16:00:00-08:00
3,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,30,30,264849,10264.0,-880,N/A,N/A,580,N/A,N/A,N/A,N/A,N/A,163.5,-110,N/A,N/A,N/A,N/A,N/A,-12,-110,N/A,N/A,12,-111,N/A,N/A,2025-11-17 16:00:00-08:00
4,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,68,68,264849,10264.0,-900,93,85,575,7,15,N/A,N/A,N/A,164.5,-112,N/A,62,63,N/A,N/A,-11.5,-115,54,70,11.5,-105,46,30,2025-11-17 16:00:00-08:00
5,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,69,69,264849,10264.0,-880,93,85,580,7,15,N/A,N/A,N/A,164.5,-110,-110,62,63,38,37,-11.5,-120,54,70,11.5,-102,46,30,2025-11-17 16:00:00-08:00
6,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,71,71,264849,10264.0,-670,93,85,440,7,15,N/A,N/A,N/A,165,-110,N/A,62,63,N/A,N/A,N/A,N/A,N/A,N/A,12,-115,46,30,2025-11-17 16:00:00-08:00
7,264849,ncaab,NC State,VCU,918,1083,complete,85,79,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,79,79,264849,10264.0,-900,93,85,600,7,15,N/A,N/A,N/A,164.5,-110,-110,62,63,38,37,-11.5,-115,54,70,11.5,-105,46,30,2025-11-17 16:00:00-08:00
8,264852,ncaab,Charleston,Drake,1048,777,complete,62,71,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,N/A,N/A,N/A,10289.0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2025-11-17 16:00:00-08:00
9,264853,ncaab,West Virginia,Lafayette,788,941,complete,81,59,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-11-18T00:00:00.000Z,15,15,264853,2948.0,-20000,90,86,3000,10,14,N/A,N/A,N/A,135,-111,-108,67,67,33,33,-23.5,-114,25,28,23.5,-105,75,72,2025-11-17 16:00:00-08:00


Appending 27 new results to ../data/ncaab_game_results.csv
Games left to play:


,game_id,match,status


...

,game_id,match,status
1,264866,Kentucky vs Michigan St,NaN
0,264881,Duke vs Kansas,NaN


Current evaluation summary:


,model,bet_payout,units,bets,ROI
0,gemini,-8.434043,16,8,-52.712767


Total Bet Payout: -51.42978911436989
Total Units: 253
Total Bets: 151
Return on Investment (ROI): -20.33%
Historical evaluation summary:


,model,bet_payout,units,bets,ROI
0,gemini,-8.434043,16,8,-52.712767
1,claude,-18.086435,117,63,-15.458492
2,perp,-24.909311,120,80,-20.757759


In [79]:
df_hist_agg = df_evaluated_hist.groupby(['model','date']).agg(
    bet_payout=('bet_payout','sum'),
    units=('units','sum'),
    bets=('game_id','count')
    ).assign(ROI=lambda x: x['bet_payout'] / x['units'] * 100).sort_values(['date','model'],ascending=True).reset_index()

df_hist_agg['CUMULATIVE_PAYOUT']=df_hist_agg.groupby('model')['bet_payout'].cumsum()
df_hist_agg['CUMULATIVE_BETS']=df_hist_agg.groupby('model')['bets'].cumsum()
df_hist_agg['CUMULATIVE_ROI'] = df_hist_agg['CUMULATIVE_PAYOUT'] / df_hist_agg['CUMULATIVE_BETS']

df_hist_agg.tail(10)

,model,date,bet_payout,units,bets,ROI,CUMULATIVE_PAYOUT,CUMULATIVE_BETS,CUMULATIVE_ROI
3,perp,2025-11-12,7.656284,19,12,40.296232,10.081711,32,0.315053
4,claude,2025-11-13,2.867205,18,8,15.928916,0.601164,30,0.020039
5,perp,2025-11-13,-2.668831,16,9,-16.680195,7.412879,41,0.180802
6,claude,2025-11-14,-4.977728,24,14,-20.740535,-4.376565,44,-0.099467
7,perp,2025-11-14,-9.558070,21,15,-45.514620,-2.145191,56,-0.038307
8,claude,2025-11-15,-9.656831,21,11,-45.984910,-14.033396,55,-0.255153
9,perp,2025-11-15,-12.375914,18,12,-68.755077,-14.521105,68,-0.213546
10,claude,2025-11-17,-4.053040,12,8,-33.775330,-18.086435,63,-0.287086
11,gemini,2025-11-17,-8.434043,16,8,-52.712767,-8.434043,8,-1.054255
12,perp,2025-11-17,-10.388206,18,12,-57.712258,-24.909311,80,-0.311366


In [80]:
fig = px.scatter(
    df_hist_agg,
    x='date',
        y='bet_payout',
        color='model',
        template='simple_white',

    )

fig.update_layout(
            # title=f"Active / Canceled Subscription Count for {customer_id}<br><sup>{viz_tag}</sup>",
            font_family='Futura',
            height=600,
            font_color='black',
            showlegend=True,
            hovermode='x unified'
        )
fig.update_traces(mode='lines+markers',
                    opacity=.75,
                    marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey'))
                    )
fig.update_xaxes(
            # title='Analytics Date',

                        )
fig.update_yaxes(
            # title='Product Count',

                        )

fig.show()

In [81]:
fig = px.scatter(
        df_hist_agg,
        x='date',
        y='CUMULATIVE_ROI',
        color='model',
        template='simple_white',

    )

fig.update_layout(
            # title=f"Active / Canceled Subscription Count for {customer_id}<br><sup>{viz_tag}</sup>",
            font_family='Futura',
            height=600,
            font_color='black',
            showlegend=True,
            hovermode='x unified'
        )
fig.update_traces(mode='lines+markers',
                    opacity=.75,
                    marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey'))
                    )
fig.update_xaxes(
            # title='Analytics Date',

                        )
fig.update_yaxes(
    tickformat = ',.0%'
            # title='Product Count',

                        )

fig.show()

In [40]:
df_evaluated_hist

,rank,model,date,game_id,match,home_score,away_score,pick,odds,units,bet_result,bet_payout
0,1,claude,2025-11-11,264701,Illinois vs Texas Tech,81,77,Over 168.5,-105,3,loss,-3.000000
1,2,claude,2025-11-11,264732,Louisville vs Kentucky,96,88,Louisville -6.5,-110,3,win,2.727273
2,3,claude,2025-11-11,264721,UNC vs Radford,89,74,UNC -19.5,-112,3,loss,-3.000000
3,4,claude,2025-11-11,266367,Michigan vs Wake Forest,85,84,Under 168.5,-110,2,loss,-2.000000
4,5,claude,2025-11-11,266649,Gonzaga vs Creighton,90,63,Over 165.5,-105,2,loss,-2.000000
5,6,claude,2025-11-11,264731,Florida vs Florida St,78,76,Florida St +17.5,-110,2,win,1.818182
6,7,claude,2025-11-11,264718,BYU vs Delaware,85,68,Over 164.5,-108,2,loss,-2.000000
7,8,claude,2025-11-11,271994,Arizona vs N. Arizona,84,49,Over 162.5,-105,2,loss,-2.000000
8,9,claude,2025-11-11,264714,Kansas vs Texas A&M-CC,77,46,Over 147.5,-105,1,loss,-1.000000
9,10,claude,2025-11-11,264709,Wisconsin vs Ball State,86,55,Ball State +27.5,-110,1,loss,-1.000000
